In [2]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [3]:
os.listdir("inside_airbnb")

['.DS_Store',
 'calendar.csv',
 'listings.csv',
 'listings_summ.csv',
 'neighbourhoods.csv',
 'neighbourhoods.geo.json',
 'neighbourhoods.geojson',
 'neighbourhoods.json',
 'reviews.csv',
 'reviews_summ.csv',
 'stations.geojson']

In [4]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [5]:
calendar['date'] = pd.to_datetime(calendar['date'])

In [6]:
calendar['month'] = calendar['date'].apply(lambda x: x.month)
calendar['day'] = calendar['date'].apply(lambda x: x.day)
calendar['holiday'] = False

In [7]:
calendar

,listing_id,date,available,price,month,day,holiday
0,11204286,2017-07-05,t,$49.00,7,5,False
1,11204286,2017-07-04,t,$49.00,7,4,False
2,11204286,2017-07-03,t,$49.00,7,3,False
3,11204286,2017-07-02,t,$49.00,7,2,False
4,11204286,2017-07-01,t,$49.00,7,1,False
5,11204286,2017-06-30,t,$49.00,6,30,False
6,11204286,2017-06-29,t,$49.00,6,29,False
7,11204286,2017-06-28,t,$49.00,6,28,False
8,11204286,2017-06-27,t,$49.00,6,27,False
9,11204286,2017-06-26,t,$49.00,6,26,False


In [25]:
calendar[calendar['month'] == 7 & calendar['day'].isin(range(21, 25))][['listing_id','price']]

,listing_id,price
146,11204286,$49.00
147,11204286,$49.00
148,11204286,$49.00
149,11204286,$49.00
493,7972006,NaN
494,7972006,NaN
495,7972006,NaN
496,7972006,NaN
859,7727710,NaN
1059,7727710,NaN


In [11]:
calendar[calendar['listing_id'] == 11204286]['price']

0      $49.00
1      $49.00
2      $49.00
3      $49.00
4      $49.00
5      $49.00
6      $49.00
7      $49.00
8      $49.00
9      $49.00
10     $49.00
11     $49.00
12     $49.00
13     $49.00
14     $49.00
15     $49.00
16     $49.00
17     $49.00
18     $49.00
19     $49.00
20     $49.00
21     $49.00
22     $49.00
23     $49.00
24     $49.00
25     $49.00
26     $49.00
27     $49.00
28     $49.00
29     $49.00
        ...  
335    $49.00
336    $49.00
337    $49.00
338    $49.00
339    $49.00
340    $49.00
341    $49.00
342    $49.00
343    $49.00
344    $49.00
345    $49.00
346    $49.00
347       NaN
348       NaN
349       NaN
350    $49.00
351    $49.00
352    $49.00
353    $49.00
354    $49.00
355    $49.00
356    $49.00
357    $49.00
358    $49.00
359    $49.00
360    $49.00
361       NaN
362    $49.00
363       NaN
364    $49.00
Name: price, Length: 365, dtype: object

In [26]:
calendar[calendar['listing_id'] == 13124681]['price']

1095    $120.00
1096    $120.00
1097    $120.00
1098    $120.00
1099    $120.00
1100    $120.00
1101    $120.00
1102    $120.00
1103    $120.00
1104    $120.00
1105    $120.00
1106    $120.00
1107    $120.00
1108    $120.00
1109    $120.00
1110    $120.00
1111    $120.00
1112    $120.00
1113    $120.00
1114    $120.00
1115    $120.00
1116    $120.00
1117    $120.00
1118    $120.00
1119    $120.00
1120    $120.00
1121    $120.00
1122    $120.00
1123    $120.00
1124    $120.00
         ...   
1430        NaN
1431        NaN
1432        NaN
1433        NaN
1434        NaN
1435        NaN
1436        NaN
1437        NaN
1438        NaN
1439        NaN
1440        NaN
1441        NaN
1442        NaN
1443        NaN
1444        NaN
1445        NaN
1446        NaN
1447        NaN
1448        NaN
1449        NaN
1450        NaN
1451        NaN
1452        NaN
1453        NaN
1454        NaN
1455        NaN
1456        NaN
1457        NaN
1458        NaN
1459        NaN
Name: price, Length: 365

In [12]:
calendar[calendar['listing_id'] == 11204286]['price'].min(), calendar[calendar['listing_id'] == 11204286]['price'].max()

(inf, '$49.00')

In [29]:
calendar['price'].isnull()[493]

True